In [34]:
import yahooquery
import pandas as pd
import numpy as np

the problem with most LSTM based stock price prediction is

In [35]:
# get previous quarters open and close prices
# never use same data twice
# always use data from before more than 1 month ago
# 3 months for a sample, predict 1 month. This gives us 4 samples per ticker per year.
# easiest additional feature to add right now is volume. 200 day MA or 200 week MA could be interesting features later.
# features must be unique to each sample (using macro data like inflation would cause model to overfit to one specific instance in history, since the data will be the same for each ticker)
# yahooquery priceHistory is sorted by date already, but add tests for this later

In [36]:
tickers = ['SPY', 'AAPL', 'SNAP', 'TSLA']
dfs = []
for ticker in tickers:
    priceHistory = yahooquery.Ticker(ticker, asnychronous=True)
    df = priceHistory.history(period='max', interval='1d')
    df = df.dropna()
    for col in ['date','high', 'low','adjclose','dividends', 'splits']:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)
    dfs.append(df)

    # TODO: if splits, separate into 2 different dfs. no samples should overlap a split.

In [37]:
samples = []
for df in dfs:
    for i in range(df['close'].count()//40): # 30 market days training, 10 market days testing
        samples.append(df.iloc[i*30:(i+1)*30])

print(len(samples))

582


In [38]:
# samplesTemp = [df.to_numpy() for df in samples]

                       open     close   volume
symbol date                                   
SPY    1993-01-29  43.96875  43.93750  1003200
       1993-02-01  43.96875  44.25000   480500
       1993-02-02  44.21875  44.34375   201300
       1993-02-03  44.40625  44.81250   529400
       1993-02-04  44.96875  45.00000   531500
                       open     close     volume
symbol date                                     
AAPL   1980-12-12  0.128348  0.128348  469033600
       1980-12-15  0.122210  0.121652  175884800
       1980-12-16  0.113281  0.112723  105728000
       1980-12-17  0.115513  0.115513   86441600
       1980-12-18  0.118862  0.118862   73449600


In [48]:

samplesTemp = [np.around(df.values, 3).astype(np.float32) for df in samples] # convert to ndarrays, reduce float64 for computation costs.
# float16 works for every stock except BRKA, but volume needs float32. test later to see if we can get away with mixed floats in matrix ops



print(samplesTemp[0][0])


X = np.vstack([arr[:30] for arr in samplesTemp]) # vstack concatenates along first axis, turning (N,) into (1,N)
Y = np.vstack([arr[30:] for arr in samplesTemp])

print(X[0])

[4.3969e+01 4.3938e+01 1.0032e+06]
[4.3969e+01 4.3938e+01 1.0032e+06]
